In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd


# 构建改进后的CNN模型
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        # 定义卷积层和批量归一化层
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)  # 定义池化层
        self.fc1 = nn.Linear(512 * 14 * 14, 1024)  # 全连接层
        self.fc2 = nn.Linear(1024, 256)  # 全连接层
        self.fc3 = nn.Linear(256, num_classes)  # 输出层，num_classes为类别数
        self.dropout = nn.Dropout(0.5)  # Dropout层，用于防止过拟合

    def forward(self, x):
        # 前向传播函数
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))  # 第一个卷积层
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))  # 第二个卷积层
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))  # 第三个卷积层
        x = self.pool(torch.relu(self.bn4(self.conv4(x))))  # 第四个卷积层
        x = x.view(-1, 512 * 14 * 14)  # 展平操作
        x = torch.relu(self.fc1(x))  # 第一个全连接层
        x = self.dropout(x)  # Dropout层
        x = torch.relu(self.fc2(x))  # 第二个全连接层
        x = self.fc3(x)  # 输出层
        return x


# 判断文件是否有效
def is_valid_file(filename):
    # 过滤掉以点开头的文件（如隐藏文件）和 .DS_Store 文件
    return not filename.startswith('.') and filename != '.DS_Store'


# 预测函数
def predict_images(model, image_dir, result_save_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 图片预处理（与训练时保持一致）
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # 调整图片大小
        transforms.ToTensor(),  # 转换为Tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
    ])

    if not os.path.exists(image_dir):
        print(f"Error: Directory {image_dir} does not exist.")
        return

    image_files = [name for name in os.listdir(image_dir) if is_valid_file(name)]
    res = ['img_name,label']  # 初始化结果文件，定义表头

    for img_name in image_files:
        img_path = os.path.join(image_dir, img_name)
        image = Image.open(img_path).convert('RGB')  # 读取图片并转换为RGB格式
        image = transform(image).unsqueeze(0).to(device)  # 预处理并转移到设备

        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs.data, 1)
            pred_class = predicted.item()
            res.append(f'{img_name},{pred_class}')

    # 将预测结果保存到result_save_path
    with open(result_save_path, 'w') as f:
        f.write('\n'.join(res))

    # 使用 Pandas 交换列的位置
    result_df = pd.read_csv(result_save_path)
    result_df = result_df[['label', 'img_name']]  # 交换列的位置
    result_df.to_csv(result_save_path, index=False)


if __name__ == "__main__":
    # 实例化模型并加载已保存的模型
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 动态选择使用 GPU 或 CPU
    num_classes = 762  # 根据检查点的类别数进行修改
    model = CNNModel(num_classes=num_classes).to(device)  # 创建模型实例

    # 加载模型参数，允许部分加载
    model_path = r'shetai\my_pytorch_model.pth'
    checkpoint = torch.load(model_path, map_location=device)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict and v.size() == model_dict[k].size()}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    model.eval()  # 设置模型为评估模式

    # 要预测的图片文件夹路径
    image_dir = r'last\shetai\image'
    # 预测结果保存文件路径
    result_save_path = r'last\shetai\jieguo.csv'

    predict_images(model, image_dir, result_save_path)

C:\Users\18089\AppData\Local\Temp\ipykernel_23444\1518687564.py:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
import pandas as pd


# 构建改进后的CNN模型
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        # 定义卷积层和批量归一化层
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)  # 定义池化层
        self.fc1 = nn.Linear(512 * 14 * 14, 1024)  # 全连接层
        self.fc2 = nn.Linear(1024, 256)  # 全连接层
        self.fc3 = nn.Linear(256, num_classes)  # 输出层，num_classes为类别数
        self.dropout = nn.Dropout(0.5)  # Dropout层，用于防止过拟合

    def forward(self, x):
        # 前向传播函数
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))  # 第一个卷积层
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))  # 第二个卷积层
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))  # 第三个卷积层
        x = self.pool(torch.relu(self.bn4(self.conv4(x))))  # 第四个卷积层
        x = x.view(-1, 512 * 14 * 14)  # 展平操作
        x = torch.relu(self.fc1(x))  # 第一个全连接层
        x = self.dropout(x)  # Dropout层
        x = torch.relu(self.fc2(x))  # 第二个全连接层
        x = self.fc3(x)  # 输出层
        return x


# 判断文件是否有效
def is_valid_file(filename):
    # 过滤掉以点开头的文件（如隐藏文件）和 .DS_Store 文件
    return not filename.startswith('.') and filename != '.DS_Store'


# 以下为逻辑函数, main函数的入参和最终的结果输出不可修改
def main(to_pred_dir, result_save_path):
    model_dir = r'shetai'

    # 实例化模型并加载已保存的模型
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 动态选择使用 GPU 或 CPU
    num_classes = 762  # 根据检查点的类别数进行修改
    model = CNNModel(num_classes=num_classes).to(device)  # 创建模型实例

    # 加载模型参数，允许部分加载
    checkpoint = torch.load(os.path.join(model_dir, 'my_pytorch_model.pth'), map_location=device)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict and v.size() == model_dict[k].size()}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    model.eval()  # 设置模型为评估模式

    # 图片预处理（与训练时保持一致）
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # 调整图片大小
        transforms.ToTensor(),  # 转换为Tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
    ])

    if not os.path.exists(to_pred_dir):
        print(f"Error: Directory {to_pred_dir} does not exist.")
        return
    print(to_pred_dir)
    # 过滤掉非文件夹的内容
    task_lst = [task for task in os.listdir(to_pred_dir) if os.path.isdir(os.path.join(to_pred_dir, task))]

    res = ['img_name,label']  # 初始化结果文件，定义表头
    for task_name in task_lst:  # 循环处理每个任务文件夹
        print(task_name)
        task_path = os.path.join(to_pred_dir, task_name)
        support_path = os.path.join(task_path, 'support')  # 支持集路径（文件夹名即为标签）
        query_path = os.path.join(task_path, 'query')  # 查询集路径（无标签，待预测图片）

        if not os.path.exists(support_path) or not os.path.exists(query_path):
            print(f"Warning: Support or query path for {task_name} does not exist.")
            continue

        prototypes = {}  # 存储各类别的原型向量
        for class_name in os.listdir(support_path):  # 加载支持集并计算原型向量
            if class_name == '.DS_Store':
                continue
            class_folder = os.path.join(support_path, class_name)
            if not os.path.isdir(class_folder):
                continue

            class_features_list = []  # 存储当前类别的特征
            for img_name in os.listdir(class_folder):  # 遍历每个类别中的图片
                if img_name.startswith('.') or img_name == '.DS_Store':
                    continue
                img_path = os.path.join(class_folder, img_name)
                image = Image.open(img_path).convert('RGB')  # 读取图片并转换为RGB格式
                image = transform(image).unsqueeze(0).to(device)  # 预处理并转移到设备

                with torch.no_grad():
                    features = model(image)  # 获取图片特征
                    class_features_list.append(features)

            if class_features_list:  # 计算该类别的原型向量（特征均值）
                class_features = torch.cat(class_features_list, dim=0)
                prototypes[class_name] = class_features.mean(dim=0)

        # 预测查询集
        test_img_lst = [name for name in os.listdir(query_path) if is_valid_file(name)]
        for img_name in test_img_lst:  # 遍历查询集中的每张图片
            name_img = os.path.join(query_path, img_name)
            image = Image.open(name_img).convert('RGB')  # 读取图片并转换为RGB格式
            image = transform(image).unsqueeze(0).to(device)  # 预处理并转移到设备

            with torch.no_grad():
                features = model(image)  # 获取图片特征
                # 计算与各类别原型向量的相似度
                similarities = {label: F.cosine_similarity(features, prototype.unsqueeze(0), dim=1).item()
                                for label, prototype in prototypes.items()}
                pred_class = max(similarities, key=similarities.get)  # 找到相似度最大的类别
                res.append(f'{img_name},{pred_class}')  # 保存结果

    # 将预测结果保存到result_save_path
    with open(result_save_path, 'w') as f:
        f.write('\n'.join(res))

    # 使用 Pandas 交换列的位置
    result_df = pd.read_csv(result_save_path)
    result_df = result_df[['label', 'img_name']]  # 交换列的位置
    result_df.to_csv(result_save_path, index=False)


if __name__ == "__main__":
    # ！！！以下内容不允许修改，修改会导致评分出错
    to_pred_dir = r'last\shetai\image'  # 所需预测的文件夹路径
    result_save_path = r'last\shetai\jg.csv'  # 预测结果保存文件路径，已指定格式为csv
    main(to_pred_dir, result_save_path)  # 调用主函数

C:\Users\18089\AppData\Local\Temp\ipykernel_23444\1394969785.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(model_dir, 'my_pytorch

D:\Work\wen_jian\中医药\last\shetai\image


In [15]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd


# 构建改进后的CNN模型
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        # 定义卷积层和批量归一化层
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)  # 定义池化层
        self.fc1 = nn.Linear(512 * 14 * 14, 1024)  # 全连接层
        self.fc2 = nn.Linear(1024, 256)  # 全连接层
        self.fc3 = nn.Linear(256, num_classes)  # 输出层，num_classes为类别数
        self.dropout = nn.Dropout(0.5)  # Dropout层，用于防止过拟合

    def forward(self, x):
        # 前向传播函数
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))  # 第一个卷积层
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))  # 第二个卷积层
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))  # 第三个卷积层
        x = self.pool(torch.relu(self.bn4(self.conv4(x))))  # 第四个卷积层
        x = x.view(-1, 512 * 14 * 14)  # 展平操作
        x = torch.relu(self.fc1(x))  # 第一个全连接层
        x = self.dropout(x)  # Dropout层
        x = torch.relu(self.fc2(x))  # 第二个全连接层
        x = self.fc3(x)  # 输出层
        return x


# 判断文件是否有效
def is_valid_file(filename):
    # 过滤掉以点开头的文件（如隐藏文件）和 .DS_Store 文件
    return not filename.startswith('.') and filename != '.DS_Store'


# 以下为逻辑函数, main函数的入参和最终的结果输出不可修改
def main(to_pred_dir, result_save_path):
    model_dir = r''

    # 实例化模型并加载已保存的模型
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 动态选择使用 GPU 或 CPU
    num_classes = 762  # 根据检查点的类别数进行修改
    model = CNNModel(num_classes=num_classes).to(device)  # 创建模型实例

    # 加载模型参数，允许部分加载
    checkpoint = torch.load(os.path.join(model_dir, 'my_pytorch_model.pth'), map_location=device)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict and v.size() == model_dict[k].size()}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    model.eval()  # 设置模型为评估模式

    # 图片预处理（与训练时保持一致）
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # 调整图片大小
        transforms.ToTensor(),  # 转换为Tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
    ])

    if not os.path.exists(to_pred_dir):
        print(f"Error: Directory {to_pred_dir} does not exist.")
        return

    # 获取目录下的所有有效图片文件
    image_files = [f for f in os.listdir(to_pred_dir) if is_valid_file(f) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    res = ['img_name,label']  # 初始化结果文件，定义表头

    for img_name in image_files:
        img_path = os.path.join(to_pred_dir, img_name)
        image = Image.open(img_path).convert('RGB')  # 读取图片并转换为RGB格式
        image = transform(image).unsqueeze(0).to(device)  # 预处理并转移到设备

        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs.data, 1)
            pred_class = predicted.item()
            res.append(f'{img_name},{pred_class}')  # 保存结果
            print(pred_class)
    # 将预测结果保存到result_save_path
    with open(result_save_path, 'w') as f:
        f.write('\n'.join(res))

    # 使用 Pandas 交换列的位置
    result_df = pd.read_csv(result_save_path)
    result_df = result_df[['label', 'img_name']]  # 交换列的位置
    result_df.to_csv(result_save_path, index=False)


if __name__ == "__main__":
    # ！！！以下内容不允许修改，修改会导致评分出错
    to_pred_dir = r'D:\Work\wen_jian\中医药\med_web\last\shetai\task1\support\Mirror-Approximated'  # 所需预测的文件夹路径
    result_save_path = r"D:\Work\wen_jian\chinese_machine\last (6)\last\shetai\jg.csv"  # 预测结果保存文件路径，已指定格式为csv
    main(to_pred_dir, result_save_path)  # 调用主函数

4
4
4
20
708
4
697
4
20
4
155
4
247
4
20
4
155
20
4
146
708
708
697
708
708
4
697
697
547
4
155
547
4
697
146
697
646
547
708
146
146
155
146
4
4
4
708
4
4
4
4
708
4
708
697
4
4
4
4
697
4
4
4
4
4
247
547
4
708
646
697
4
4
20
4
4
697
697
697
416
4
155
697
697
4
146
4
697
697
4
247
4
697
708
4
416
416
4
697
697
4
20
697
4
4
708
697
20
4
4
247
708


PermissionError: [Errno 13] Permission denied: 'D:\\Work\\wen_jian\\chinese_machine\\last (6)\\last\\shetai\\jg.csv'

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
import pandas as pd


# 构建改进后的CNN模型
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        # 定义卷积层和批量归一化层
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2, 2)  # 定义池化层
        self.fc1 = nn.Linear(512 * 14 * 14, 1024)  # 全连接层
        self.fc2 = nn.Linear(1024, 256)  # 全连接层
        self.fc3 = nn.Linear(256, num_classes)  # 输出层，num_classes为类别数
        self.dropout = nn.Dropout(0.5)  # Dropout层，用于防止过拟合

    def forward(self, x):
        # 前向传播函数
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))  # 第一个卷积层
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))  # 第二个卷积层
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))  # 第三个卷积层
        x = self.pool(torch.relu(self.bn4(self.conv4(x))))  # 第四个卷积层
        x = x.view(-1, 512 * 14 * 14)  # 展平操作
        x = torch.relu(self.fc1(x))  # 第一个全连接层
        x = self.dropout(x)  # Dropout层
        x = torch.relu(self.fc2(x))  # 第二个全连接层
        x = self.fc3(x)  # 输出层
        return x


# 判断文件是否有效
def is_valid_file(filename):
    # 过滤掉以点开头的文件（如隐藏文件）和 .DS_Store 文件
    return not filename.startswith('.') and filename != '.DS_Store'


# 以下为逻辑函数, main函数的入参和最终的结果输出不可修改
def main(to_pred_dir, result_save_path):
    model_dir = r'shetai'

    # 实例化模型并加载已保存的模型
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 动态选择使用 GPU 或 CPU
    num_classes = 762  # 根据检查点的类别数进行修改
    model = CNNModel(num_classes=num_classes).to(device)  # 创建模型实例

    # 加载模型参数，允许部分加载
    checkpoint = torch.load(os.path.join(model_dir, 'my_pytorch_model.pth'), map_location=device)
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict and v.size() == model_dict[k].size()}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    model.eval()  # 设置模型为评估模式

    # 图片预处理（与训练时保持一致）
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # 调整图片大小
        transforms.ToTensor(),  # 转换为Tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
    ])

    task1_support_path = os.path.join(to_pred_dir, 'task1\support')
    if not os.path.exists(task1_support_path):
        print(f"Error: Directory {task1_support_path} does not exist.")
        return

    prototypes = {}  # 存储各类别的原型向量
    for class_name in os.listdir(task1_support_path):  # 加载支持集并计算原型向量
        if class_name == '.DS_Store':
            continue
        class_folder = os.path.join(task1_support_path, class_name)
        if not os.path.isdir(class_folder):
            continue

        class_features_list = []  # 存储当前类别的特征
        for img_name in os.listdir(class_folder):  # 遍历每个类别中的图片
            if img_name.startswith('.') or img_name == '.DS_Store':
                continue
            img_path = os.path.join(class_folder, img_name)
            image = Image.open(img_path).convert('RGB')  # 读取图片并转换为RGB格式
            image = transform(image).unsqueeze(0).to(device)  # 预处理并转移到设备

            with torch.no_grad():
                features = model(image)  # 获取图片特征
                class_features_list.append(features)

        if class_features_list:  # 计算该类别的原型向量（特征均值）
            class_features = torch.cat(class_features_list, dim=0)
            prototypes[class_name] = class_features.mean(dim=0)

    if not os.path.exists(to_pred_dir):
        print(f"Error: Directory {to_pred_dir} does not exist.")
        return

    # 获取目录下的所有有效图片文件
    image_files = [f for f in os.listdir(to_pred_dir) if is_valid_file(f) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    res = ['img_name,label']  # 初始化结果文件，定义表头

    for img_name in image_files:
        img_path = os.path.join(to_pred_dir, img_name)
        image = Image.open(img_path).convert('RGB')  # 读取图片并转换为RGB格式
        image = transform(image).unsqueeze(0).to(device)  # 预处理并转移到设备

        with torch.no_grad():
            features = model(image)  # 获取图片特征
            # 计算与各类别原型向量的相似度
            similarities = {label: F.cosine_similarity(features, prototype.unsqueeze(0), dim=1).item()
                            for label, prototype in prototypes.items()}
            pred_class = max(similarities, key=similarities.get)  # 找到相似度最大的类别
            res.append(f'{img_name},{pred_class}')  # 保存结果

    # 将预测结果保存到result_save_path
    with open(result_save_path, 'w') as f:
        f.write('\n'.join(res))

    # 使用 Pandas 交换列的位置
    result_df = pd.read_csv(result_save_path)
    result_df = result_df[['label', 'img_name']]  # 交换列的位置
    result_df.to_csv(result_save_path, index=False)


if __name__ == "__main__":
    # ！！！以下内容不允许修改，修改会导致评分出错
    to_pred_dir = r'last\shetai'  # 所需预测的文件夹路径
    result_save_path = r'last\shetai\jg.csv'  # 预测结果保存文件路径，已指定格式为csv
    main(to_pred_dir, result_save_path)

C:\Users\18089\AppData\Local\Temp\ipykernel_15352\580647341.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(model_dir, 'my_pytorch_

In [7]:
import torch
from torch import nn
from torchvision import transforms

def load_model(model_path, num_classes=762):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 构建改进后的CNN模型
    class CNNModel(nn.Module):
        def __init__(self, num_classes):
            super(CNNModel, self).__init__()
            # 定义卷积层和批量归一化层
            self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
            self.bn1 = nn.BatchNorm2d(64)
            self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
            self.bn2 = nn.BatchNorm2d(128)
            self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
            self.bn3 = nn.BatchNorm2d(256)
            self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
            self.bn4 = nn.BatchNorm2d(512)
            self.pool = nn.MaxPool2d(2, 2)  # 定义池化层
            self.fc1 = nn.Linear(512 * 14 * 14, 1024)  # 全连接层
            self.fc2 = nn.Linear(1024, 256)  # 全连接层
            self.fc3 = nn.Linear(256, num_classes)  # 输出层，num_classes为类别数
            self.dropout = nn.Dropout(0.5)  # Dropout层，用于防止过拟合

        def forward(self, x):
            # 前向传播函数
            x = self.pool(torch.relu(self.bn1(self.conv1(x))))  # 第一个卷积层
            x = self.pool(torch.relu(self.bn2(self.conv2(x))))  # 第二个卷积层
            x = self.pool(torch.relu(self.bn3(self.conv3(x))))  # 第三个卷积层
            x = self.pool(torch.relu(self.bn4(self.conv4(x))))  # 第四个卷积层
            x = x.view(-1, 512 * 14 * 14)  # 展平操作
            x = torch.relu(self.fc1(x))  # 第一个全连接层
            x = self.dropout(x)  # Dropout层
            x = torch.relu(self.fc2(x))  # 第二个全连接层
            x = self.fc3(x)  # 输出层
            return x


    # 加载模型
    model = CNNModel(num_classes).to(device)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint)
    model.eval()
    return model, device
transform = transforms.Compose([
        transforms.Resize((224, 224)),  # 调整图片大小
        transforms.ToTensor(),  # 转换为Tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
    ])

# 预测函数
def predict_image(image_path, model, device, prototypes):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # 预处理
    with torch.no_grad():
        features = model(image)  # 提取特征
        # 计算与各类别原型向量的相似度
        similarities = {
            label: torch.cosine_similarity(features, proto.unsqueeze(0), dim=1).item()
            for label, proto in prototypes.items()
        }
    return max(similarities, key=similarities.get)  # 返回最相似的类别

In [5]:
from django.http import JsonResponse, HttpResponse
import json
from django.urls import reverse
from django.views.decorators.csrf import csrf_exempt
import base64
from io import BytesIO
from PIL import Image
from django.shortcuts import render, redirect
from django.core.files.base import ContentFile
import os
from django.conf import settings
import datetime
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd

model_path = "my_pytorch_model.pth"  # 确保路径正确
support_dir = "task1/support"        # 确保路径正确

# 加载模型
model, device = load_model(model_path,num_classes=762)

# 计算原型向量
prototypes = {}
for class_name in os.listdir(support_dir):
    if class_name == '.DS_Store':
        continue
    class_folder = os.path.join(support_dir, class_name)
    if not os.path.isdir(class_folder):
        continue

    class_features_list = []
    for img_name in os.listdir(class_folder):
        if img_name.startswith('.') or img_name == '.DS_Store':
            continue
        img_path = os.path.join(class_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            features = model(image)
            class_features_list.append(features)

    if class_features_list:
        class_features = torch.cat(class_features_list, dim=0)
        prototypes[class_name] = class_features.mean(dim=0)

RuntimeError: Error(s) in loading state_dict for CNNModel:
	size mismatch for fc3.weight: copying a param with shape torch.Size([5, 256]) from checkpoint, the shape in current model is torch.Size([762, 256]).
	size mismatch for fc3.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([762]).

In [14]:
import torch

# 保存到文件
torch.save(prototypes, "prototypes.pt")

# 之后可以加载
loaded_prototypes = torch.load("prototypes.pt")
print(loaded_prototypes.keys())  # 查看所有类别

dict_keys(['Grey-Black', 'Mirror-Approximated', 'Thin-White', 'White-Greasy', 'Yellow-Greasy'])


In [8]:
from django.http import JsonResponse, HttpResponse
import json
from django.urls import reverse
from django.views.decorators.csrf import csrf_exempt
import base64
from io import BytesIO
from PIL import Image
from django.shortcuts import render, redirect
from django.core.files.base import ContentFile
import os
from django.conf import settings
import datetime
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd

# 初始化
model_path = "my_pytorch_model.pth"
prototypes_path = "prototypes.pt"  # 你保存的原型向量文件
model, device = load_model(model_path,num_classes=762)
# 加载模型和原型向量

prototypes = torch.load(prototypes_path, map_location=device)  # 直接加载

# 预测新图像
result = predict_image(r"D:\Work\wen_jian\中医药\med_web\last\shetai\task1\support\White-Greasy\White-Greasy_646.jpg", model, device, prototypes)
print(f"预测结果: {result}")

RuntimeError: Error(s) in loading state_dict for CNNModel:
	size mismatch for fc3.weight: copying a param with shape torch.Size([5, 256]) from checkpoint, the shape in current model is torch.Size([762, 256]).
	size mismatch for fc3.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([762]).